In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp downloaderlabeler

In [ ]:
!pip install .[dev]

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
import pickle
from steroidsornot.pushshift import select_mostly_useful, unpickle

In [ ]:
len(select_mostly_useful(downloader_labeler.pickled_posts))

In [ ]:
unpickle('data/all_nattyorjuice_submissions.pkl')

In [ ]:
class DownloaderLabeler():
    def __init__(self, path):
        self.pickled_posts = []
        self._unpickle(path)
        
        self._setup_buttons()
        

        
        self._main_loop()
        
    def _main_loop(self):
        for submission in self.pickled_posts
        
    def _next_submission():
        

        
    def _setup_buttons(self):
        self.natural_button = Button(description='natural', layout=Layout(border = '2px solid green'))
        self.uncertain_button = Button(description='uncertain', layout=Layout(border = '2px solid brown'))
        self.steroids_button = Button(description='steroids', layout=Layout(border = '2px solid red'))
        self.irrelevant_button = Button(description='irrelevant', layout=Layout(border = '2px solid khaki'))
#         display(self.natural_button, self.steroids_button)
        self.box = widgets.HBox([self.natural_button,
                                 self.uncertain_button,
                                 self.steroids_button,
                                 self.irrelevant_button,
                                ])
        display(self.box)
        
        for button in self.box
        

In [ ]:
downloader_labeler = DownloaderLabeler('data/downloader.pkl')